In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.dimension_users
USING DELTA
AS
SELECT
  time,
  submit_id,
  submit_datetime,
  review_id,
  review_datetime,
  approval_id,
  approval_datetime,
  status
FROM workspace.gold_staging.stg_users;

MERGE INTO workspace.gold_serving.dimension_users AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      submit_id,
      submit_datetime,
      review_id,
      review_datetime,
      approval_id,
      approval_datetime,
      status,
      ROW_NUMBER() OVER (
        PARTITION BY time, submit_id -- or other unique keys
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_users
  ) WHERE rn = 1
) AS source
ON target.submit_id = source.submit_id
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.submit_datetime = source.submit_datetime,
    target.review_id = source.review_id,
    target.review_datetime = source.review_datetime,
    target.approval_id = source.approval_id,
    target.approval_datetime = source.approval_datetime,
    target.status = source.status
WHEN NOT MATCHED THEN
  INSERT (
    time,
    submit_id,
    submit_datetime,
    review_id,
    review_datetime,
    approval_id,
    approval_datetime,
    status
  )
  VALUES (
    source.time,
    source.submit_id,
    source.submit_datetime,
    source.review_id,
    source.review_datetime,
    source.approval_id,
    source.approval_datetime,
    source.status
  );

